This exercise will require you to pull some data from https://data.nasdaq.com/ (formerly Quandl API).

As a first step, you will need to register a free account on the https://data.nasdaq.com/ website.

After you register, you will be provided with a unique API key, that you should store:

*Note*: Use a `.env` file and put your key in there and `python-dotenv` to access it in this notebook. 

The code below uses a key that was used when generating this project but has since been deleted. Never submit your keys to source control. There is a `.env-example` file in this repository to illusrtate what you need. Copy that to a file called `.env` and use your own api key in that `.env` file. Make sure you also have a `.gitignore` file with a line for `.env` added to it. 

The standard Python gitignore is [here](https://github.com/github/gitignore/blob/master/Python.gitignore) you can just copy that. 

In [172]:
# get api key from your .env file
import os
from dotenv import load_dotenv

load_dotenv()
API_KEY = os.getenv('NASDAQ_API_KEY')

print(API_KEY)

voQmznZhzRD5QdAAx6_k


Nasdaq Data has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Nasdaq Data API instructions here: https://docs.data.nasdaq.com/docs/in-depth-usage

While there is a dedicated Python package for connecting to the Nasdaq API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [173]:
# First, import the relevant modules

import json
import requests

Note: API's can change a bit with each version, for this exercise it is reccomended to use the nasdaq api at `https://data.nasdaq.com/api/v3/`. This is the same api as what used to be quandl so `https://www.quandl.com/api/v3/` should work too.
Hint: We are looking for the `AFX_X` data on the `datasets/FSE/` dataset.

In [174]:
# Now, call the Nasdaq API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
u = 'https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X'
r = requests.get(url)
print(r)
j_data = r.json()

<Response [200]>


In [175]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
print(j_data.keys())
print(j_data['dataset'].keys())

dict_keys(['dataset'])
dict_keys(['id', 'dataset_code', 'database_code', 'name', 'description', 'refreshed_at', 'newest_available_date', 'oldest_available_date', 'column_names', 'frequency', 'type', 'premium', 'limit', 'transform', 'column_index', 'start_date', 'end_date', 'data', 'collapse', 'order', 'database_id'])


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [176]:
url = 'https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X.json?order=asc&start_date=2016-12-31&end_date=2017-12-31'
result = requests.get(url)
print(result)
data = result.json()
print(data['dataset'].keys())

<Response [200]>
dict_keys(['id', 'dataset_code', 'database_code', 'name', 'description', 'refreshed_at', 'newest_available_date', 'oldest_available_date', 'column_names', 'frequency', 'type', 'premium', 'limit', 'transform', 'column_index', 'start_date', 'end_date', 'data', 'collapse', 'order', 'database_id'])


In [177]:
print(data['dataset']['column_names'])

['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover']


In [178]:
date_max = ''
date_min = ''
max = data['dataset']['data'][0][1]
min = data['dataset']['data'][0][1]
for list in data['dataset']['data']:
    if type(list[1]) == type(1.1):
        if max < list[1]:
            max = list[1]
            date_max = list[0]
        if min > list[1]:
            min = list[1]
            date_min = list[0]

print(max, date_max)
print(min, date_min)

53.11 2017-12-14
34.0 2017-01-24


In [179]:
date = ''
max_swing = data['dataset']['data'][0][2]-data['dataset']['data'][0][3]
for list in data['dataset']['data']:
    if type(list[1]) == type(1.1):
        if max_swing < list[2]-list[3]:
            max_swing = list[2]-list[3]
            date = list[0]
print(max_swing, date)

max_change = data['dataset']['data'][0][1]-data['dataset']['data'][0][4]
for list in data['dataset']['data']:
    if type(list[1]) == type(1.1):
        if abs(max_change) < abs(list[1]-list[4]):
            max_change = abs(list[1]-list[4])
            date = list[0]
print(max_change, date)

2.8100000000000023 2017-05-11
1.6899999999999977 2017-08-09


In [180]:
trade_volume = 0
c = 0

for list in data['dataset']['data']:
    if type(list[6]) == type(1.1):
        trade_volume += list[6]
        c += 1
    
print(trade_volume/c)

89124.33725490196
